In [1]:
import pandas as pd

# Verileri yükle
train = pd.read_csv("train_new.csv")
test = pd.read_csv("test_new.csv")

# İlk bakış
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Train columns:", train.columns.tolist())
print("Test columns:", test.columns.tolist())

# Eksik veri analizi
print("\nEksik değer sayısı (train):\n", train.isnull().sum())
print("\nEksik değer sayısı (test):\n", test.isnull().sum())


Train shape: (150826, 15)
Test shape: (37707, 14)
Train columns: ['id', 'brand', 'model', 'year', 'km', 'fuel_type', 'transmission', 'color_ext', 'color_int', 'accident', 'clean_title', 'price', 'horsepower', 'engine_liter', 'age']
Test columns: ['id', 'brand', 'model', 'year', 'km', 'fuel_type', 'transmission', 'color_ext', 'color_int', 'accident', 'clean_title', 'horsepower', 'engine_liter', 'age']

Eksik değer sayısı (train):
 id                  0
brand               0
model               0
year                0
km                  0
fuel_type        4060
transmission        0
color_ext           0
color_int           0
accident         1967
clean_title     17058
price               0
horsepower      26575
engine_liter    11343
age                 0
dtype: int64

Eksik değer sayısı (test):
 id                 0
brand              0
model              0
year               0
km                 0
fuel_type       1023
transmission       0
color_ext          0
color_int          0
accid

In [2]:
import pandas as pd

# Verileri oku
train = pd.read_csv("train_new.csv")
test = pd.read_csv("test_new.csv")

# Price sütununu geçici olarak ayır
train_price = train["price"]

# 'price' sütunu olmadan birleştir
combined = pd.concat([train.drop(columns=["price"]), test], axis=0).reset_index(drop=True)

# 🔧 Sayısal sütunlardaki eksik değerleri doldur
for col in ["horsepower", "engine_liter"]:
    mean_value = combined[col].mean()
    combined[col] = combined[col].fillna(mean_value)

# 🔧 Kategorik sütunlardaki eksik değerleri doldur
for col in ["fuel_type", "transmission", "color_int", "accident", "clean_title"]:
    combined[col] = combined[col].fillna("Unknown")

# 💡 Not: Burada 'accident' ve 'clean_title' kategorik olarak sayıldıkları için "Unknown" veriyoruz.

# 🔁 Verileri tekrar ayır
train_cleaned = combined.iloc[:len(train)].copy()
test_cleaned = combined.iloc[len(train):].copy()

# 'price' sütununu geri ekle
train_cleaned.loc[:, "price"] = train_price


In [3]:
# Sayısal sütunları ortalama ile doldur
numeric_cols = ["horsepower", "engine_liter"]
for col in numeric_cols:
    mean_value = combined[col].mean()
    combined[col] = combined[col].fillna(mean_value)
    print(f"{col}: eksikler ortalama ({mean_value:.2f}) ile dolduruldu.")


horsepower: eksikler ortalama (343.26) ile dolduruldu.
engine_liter: eksikler ortalama (3.81) ile dolduruldu.


In [4]:
categorical_cols = ["fuel_type", "transmission", "color_int", "clean_title"]
for col in categorical_cols:
    combined[col] = combined[col].fillna("Unknown")
    print(f"{col}: eksikler 'Unknown' ile dolduruldu.")


fuel_type: eksikler 'Unknown' ile dolduruldu.
transmission: eksikler 'Unknown' ile dolduruldu.
color_int: eksikler 'Unknown' ile dolduruldu.
clean_title: eksikler 'Unknown' ile dolduruldu.


In [5]:
print("\nVeri tipleri kontrolü:\n", combined.dtypes)



Veri tipleri kontrolü:
 id                int64
brand            object
model            object
year              int64
km                int64
fuel_type        object
transmission     object
color_ext        object
color_int        object
accident         object
clean_title      object
horsepower      float64
engine_liter    float64
age               int64
dtype: object


In [6]:
# Satır sayısıyla tekrar ayır
train_cleaned = combined.iloc[:len(train)].copy()
test_cleaned = combined.iloc[len(train):].copy()

# 'price' sütununu tekrar ekle
train_cleaned["price"] = train["price"]


In [7]:
print("\nTrain Cleaned eksik değer kontrolü:\n", train_cleaned.isnull().sum())
print("\nTest Cleaned eksik değer kontrolü:\n", test_cleaned.isnull().sum())



Train Cleaned eksik değer kontrolü:
 id              0
brand           0
model           0
year            0
km              0
fuel_type       0
transmission    0
color_ext       0
color_int       0
accident        0
clean_title     0
horsepower      0
engine_liter    0
age             0
price           0
dtype: int64

Test Cleaned eksik değer kontrolü:
 id              0
brand           0
model           0
year            0
km              0
fuel_type       0
transmission    0
color_ext       0
color_int       0
accident        0
clean_title     0
horsepower      0
engine_liter    0
age             0
dtype: int64


In [8]:
train_cleaned.to_csv("train_cleaned.csv", index=False)
test_cleaned.to_csv("test_cleaned.csv", index=False)
print("\nTemizlenmiş veriler kaydedildi: train_cleaned.csv ve test_cleaned.csv")
print("\nİşlem tamamlandı. Veriler temizlendi ve kaydedildi.")


Temizlenmiş veriler kaydedildi: train_cleaned.csv ve test_cleaned.csv

İşlem tamamlandı. Veriler temizlendi ve kaydedildi.


In [9]:
print(train_cleaned["price"].describe())
print(train_cleaned["horsepower"].describe())


count    1.508260e+05
mean     4.389079e+04
std      7.984588e+04
min      2.000000e+03
25%      1.700000e+04
50%      3.079800e+04
75%      4.989500e+04
max      2.954083e+06
Name: price, dtype: float64
count    150826.000000
mean        343.252052
std         104.278503
min          70.000000
25%         280.000000
50%         343.263856
75%         395.000000
max        1020.000000
Name: horsepower, dtype: float64


In [10]:
print(train_cleaned["fuel_type"].value_counts())


fuel_type
Gasoline          132832
Hybrid              5437
E85 Flex Fuel       4328
Unknown             4060
Diesel              3101
–                    625
Plug-In Hybrid       433
not supported         10
Name: count, dtype: int64


In [11]:
# Ortalama değeri yeniden hesaplayalım (emin olmak için)
hp_mean = train_cleaned["horsepower"].mean()
el_mean = train_cleaned["engine_liter"].mean()

# Eksik olanları işaretle
train_cleaned["horsepower_missing"] = (train_cleaned["horsepower"] == hp_mean).astype(int)
test_cleaned["horsepower_missing"] = (test_cleaned["horsepower"] == hp_mean).astype(int)

train_cleaned["engine_liter_missing"] = (train_cleaned["engine_liter"] == el_mean).astype(int)
test_cleaned["engine_liter_missing"] = (test_cleaned["engine_liter"] == el_mean).astype(int)


In [12]:
print(train_cleaned[["horsepower_missing", "engine_liter_missing"]].head())


   horsepower_missing  engine_liter_missing
0                   0                     0
1                   0                     0
2                   0                     0
3                   0                     0
4                   0                     0


In [13]:
train_cleaned.to_csv("train_cleaned_v2.csv", index=False)
test_cleaned.to_csv("test_cleaned_v2.csv", index=False)
print("Yeni flag'lerle birlikte temizlenmiş veriler kaydedildi.")


Yeni flag'lerle birlikte temizlenmiş veriler kaydedildi.


In [3]:
import pandas as pd

solution = pd.read_csv("solution.csv")

# Sıralama ve kullanım şekline göre Usage sütunu ekle
# Örnek olarak %50'si public, %50'si private diyelim:
usage = ["Public"] * (len(solution) // 2) + ["Private"] * (len(solution) - len(solution) // 2)

solution.insert(0, "Usage", usage)

solution.to_csv("solution_with_usage.csv", index=False)

In [4]:
solution.to_csv("solution_with_usage.csv", index=False)
print("Çözüm dosyasına Usage sütunu eklendi ve kaydedildi: solution_with_usage.csv")

Çözüm dosyasına Usage sütunu eklendi ve kaydedildi: solution_with_usage.csv


In [2]:
import pandas as pd

# Orijinal çözüm dosyasını oku
solution = pd.read_csv("solution_with_usage.csv")

# Örnek olarak tüm satırları Public yap
solution["Usage"] = ["Public"] * (len(solution) // 2) + ["Private"] * (len(solution) - len(solution) // 2)

# Kolon sırasını düzenle: id, Usage, price
solution = solution[["id", "Usage", "price"]]

# Yeni dosyayı kaydet
solution.to_csv("solution_corrected.csv", index=False)


In [2]:
import pandas as pd

# Var olan çözüm dosyasını yükle
solution = pd.read_csv("solution_corrected.csv")

# Satır sayısını al
total_rows = len(solution)
public_count = int(total_rows * 0.2)
private_count = total_rows - public_count

# Usage kolonunu oluştur
solution["Usage"] = ["Public"] * public_count + ["Private"] * private_count

# Sütun sırasını yeniden düzenle
solution = solution[["id", "Usage", "price"]]

# Yeni dosyayı kaydet
solution.to_csv("solution_20_80.csv", index=False)